In [1]:
import pandas as pd
import csv
from loader import CreateMovieBaskets, ReadMovies
from pair_finder import *
import time

In [2]:
ratings_path = "DATA\\ratings_100users.csv"
movies_path =  "DATA\\movies.csv"
MinScore = 4.5 # CHANGE THIS TO USER INPUT

UserBaskets,movies_list = CreateMovieBaskets(ratings_path,MinScore, limit=20)
movies_df = ReadMovies(movies_path)
movies_df = movies_df[movies_df['movieId'].isin(movies_list)].reset_index(drop=True)



In [3]:
ub = [
    [0,1,2],
    [1,2,3],
    [0,1,3],
    [1,3,4]
]
for i in ub:
    print(get_pairs(i))

['1,0', '2,0', '2,1']
['2,1', '3,1', '3,2']
['1,0', '3,0', '3,1']
['3,1', '4,1', '4,3']


In [4]:
def apriori(movies_df, itemBaskets, min_frequency, min_length):
    C1 = calculate_frequencies(itemBaskets)
    L1 = frequency_filter(min_frequency, C1)
    
    



In [4]:
def calculate_frequencies(object_list, N):
    vector_dict = {}

    for b_i in range(len(object_list)):
        for c in object_list[b_i]: # c -> combo
            if c not in vector_dict.keys():
                vector_dict[c] = {"frequency": 1, "baskets" : [b_i]}
            else:
                vector_dict[c]['frequency'] += 1
                vector_dict[c]['baskets'].append(b_i)
    for i in vector_dict.keys():
        vector_dict[i]['frequency'] = vector_dict[i]['frequency'] / N
    
    return vector_dict
    
def frequency_filter(min_frequency, input_dict):
    
    for i in list(input_dict):
        if input_dict[i]['frequency'] < min_frequency:
            del input_dict[i]
    
    return input_dict

In [64]:
fd = calculate_frequencies(ub, len(ub))
print(fd)
frequency_filter(0.5, fd)

{0: {'frequency': 0.5, 'baskets': [0, 2]}, 1: {'frequency': 1.0, 'baskets': [0, 1, 2, 3]}, 2: {'frequency': 0.5, 'baskets': [0, 1]}, 3: {'frequency': 0.75, 'baskets': [1, 2, 3]}, 4: {'frequency': 0.25, 'baskets': [3]}}


{0: {'frequency': 0.5, 'baskets': [0, 2]},
 1: {'frequency': 1.0, 'baskets': [0, 1, 2, 3]},
 2: {'frequency': 0.5, 'baskets': [0, 1]},
 3: {'frequency': 0.75, 'baskets': [1, 2, 3]}}

In [26]:
tmp_list  = [1,2,5,7,9,10]

while len(tmp) != 1:
    for i in range(1,len(tmp)):
        for j in range(i):
            print()

[0, 1, 2]

In [39]:
tmp = [[1,2,3], [1,2,3], [0,2,3]]
'''for i in range(len(tmp)):
    tmp[i] = [f'{j}'for j in tmp[i]]
tmp1 = []
tmp1 = [tmp[i] for i in range(len(tmp)) if i == 0 or tmp[i] != tmp[i-1]] # REMOVES DUPLICATES FROM LIST OF LISTS
tmp1'''

[['1', '2', '3'], ['0', '2', '3']]

In [40]:
tmp = [[1,2,3], [1,2,3], [0,2,3]]
pairs = []
for i in tmp:
    pairs.append(get_pairs(i))

In [50]:
unique_pairs=[]
for i in pairs:
    for j in i:
        if j not in unique_pairs:
            unique_pairs.append(j)
unique_pairs

[(2, 1), (3, 1), (3, 2), (2, 0), (3, 0)]

In [51]:
unique_pairs = [ f"{i[0]},{i[-1]}" for i in unique_pairs]
unique_pairs

['2,1', '3,1', '3,2', '2,0', '3,0']

In [54]:
def get_combos(ub, L, k):
    combo_list = []
    for basket in ub:
        if len(basket) >= k:
            for i in basket:
                if i in list(L[1]):
                    for j in list(L[k-1]):
                        if i not in j  and j in basket:
                        

1 3,2
BADINGA
1 2,0
BADINGA
1 3,0
BADINGA
2 3,1
BADINGA
2 3,0
BADINGA
3 2,1
BADINGA
3 2,0
BADINGA


In [10]:
L = {}
C1 = calculate_frequencies(ub ,len(ub))
L[1] = frequency_filter(0.5,C1)
pairs = []
for i in ub:
    pairs.append(get_pairs(i))
C2 = calculate_frequencies(pairs, len(ub))
L[2] = frequency_filter(0.5, C2)
L

{1: {0: {'frequency': 0.5, 'baskets': [0, 2]},
  1: {'frequency': 1.0, 'baskets': [0, 1, 2, 3]},
  2: {'frequency': 0.5, 'baskets': [0, 1]},
  3: {'frequency': 0.75, 'baskets': [1, 2, 3]}},
 2: {'1,0': {'frequency': 0.5, 'baskets': [0, 2]},
  '2,1': {'frequency': 0.5, 'baskets': [0, 1]},
  '3,1': {'frequency': 0.75, 'baskets': [1, 2, 3]}}}